In [1]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 9.7 MB/s eta 0:00:00


In [2]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048 # choosing inputs max tokens  context window llama 128k, llama3.2's embedding size is 3072 first input layer also 3072,  ffn expansion 2.7x 8192 ,attention head 24in number. each learns its qkv and attention score , each dimension by attention head 128,gqa,  kv head 8
dtype = None # None for auto detection.
load_in_4bit = True # Using 4bit quantization to reduce memory usage.I can put it to false also


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [23]:
#getting peft and defining paramweters for LORA

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #Hyperparameter
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], #llama has a exclusive gate proj  eg  , wq plus delta w adapter
    #x>>W1up, w2up, >>xw1up, activations (xw2up),>>>w down


   lora_alpha = 16, # a higher alpha value will assign more weight to the LoRA activations  alpha/rank which is delata  a scaling paramter
   lora_dropout = 0, #dropout regularization
    bias = "none",    #I can also keep it all or lora_only
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 1997,
    use_rslora = False,# my rank stabilised lora in case lora alpha is unstable
    loftq_config = None#lora and qlora fusion wehn i am quantizing its learn low rank adaption simuletaneously for better accuracy
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [24]:
from datasets import load_dataset
dataset = load_dataset("ServiceNow-AI/R1-Distill-SFT",'v0', split = "train")

#https://huggingface.co/datasets/ServiceNow-AI/R1-Distill-SFT

In [15]:
print(dataset[:5])

{'id': ['id_0', 'id_1', 'id_2', 'id_3', 'id_4'], 'reannotated_assistant_content': ['<think>\nFirst, I need to determine the total number of children on the playground by adding the number of boys and girls.\n\nThere are 27 boys and 35 girls.\n\nAdding these together: 27 boys + 35 girls = 62 children.\n\nTherefore, the total number of children on the playground is 62.\n</think>\n\nTo find the total number of children on the playground, we simply add the number of boys and girls together.\n\n\\[\n\\text{Total children} = \\text{Number of boys} + \\text{Number of girls}\n\\]\n\nPlugging in the given values:\n\n\\[\n\\text{Total children} = 27 \\text{ boys} + 35 \\text{ girls} = 62 \\text{ children}\n\\]\n\n**Final Answer:**\n\n\\[\n\\boxed{62}\n\\]', '<think>\nFirst, I need to determine the cost per dozen oranges. John bought three dozen oranges for \\$28.80, so I can find the cost per dozen by dividing the total cost by the number of dozens.\n\nNext, with the cost per dozen known, I can 

In [25]:
#Now creating a prompt  that i will  use to finetune our Llama model
r1_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>

{}
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  problems = examples["problem"]
  thoughts = examples["reannotated_assistant_content"]
  solutions = examples["solution"]
  texts = []

  for problem, thought, solution in zip(problems, thoughts, solutions):
    text = r1_prompt.format(problem, thought, solution)+EOS_TOKEN
    texts.append(text)

  return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True,)





Map:   0%|          | 0/171647 [00:00<?, ? examples/s]

In [27]:
#creating a trainer object from transformer reinforcement library from hugging face


from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,  #that LoRA model
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = False, # It can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 5, # Few updates with low learning rate before actual training
        max_steps = 60, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,# controlling gradient update step
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,# L2 regulirazrion discourages huge weights
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",#directory where it will be stored
        report_to = "none", # can be used  for obervability in tensorboard
    ),
)

#2x4x60 480 examples
#low enropy model better ate predicting next token

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/171647 [00:00<?, ? examples/s]

In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 171,647 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
1,1.010000,0
2,0.935000,No Log
3,1.034100,No Log
4,0.943600,No Log
5,0.785500,No Log
6,0.853200,No Log
7,0.756000,No Log
8,0.741200,No Log
9,0.786000,No Log
10,0.736400,No Log


In [29]:
import math

final_loss = trainer_stats.training_loss  # average loss across training
perplexity = math.exp(final_loss)
print(f"Final loss: {final_loss:.4f}, Perplexity: {perplexity:.4f}")

Final loss: 0.6264, Perplexity: 1.8708


In [ ]:
# MY TAKEAWAYS


# In LLM fine-tuning, the training objective is next-token prediction, not numeric regression.

# Therefore, loss values don’t approach zero( close to 0.5 is good enough depends on guidelines) — instead, they reflect how well the model predicts tokens from a large vocabulary.

# In my run, the model achieved:

# Final Loss: ~0.6264

# Perplexity : ~1.87

# A loss of ~0.6 indicates the model is learning the dataset’s reasoning patterns effectively.

# A perplexity of ~1.87 means that, on average, the model is choosing between fewer than 2 plausible tokens at each step → showing high confidence.

# Unlike regression tasks, the goal here isn’t driving loss toward zero, but ensuring a steady downward trend in loss and perplexity, paired with qualitative improvements in reasoning outputs.

In [ ]:
#INFERENCE

In [34]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>
"""
message = sys_prompt.format("If its raining why is the sky grey")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enables 2x faster inference

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)  #batch decode helps me to convert ids back into natural language so that i can see

In [35]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
If its raining why is the sky grey
</problem>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hmmm, okay, let me try to understand this. The question is: "If it's raining, why is the sky grey." Hmm, that seems like a pretty straightforward question, but maybe there's more to it. Maybe the sky is grey when it's raining because of how water in the air behaves.

Alright, so first, I remember learning about different types of clouds and how they form. Clouds are made up of water vapor and tiny particles in the air. When the sun heats the Earth, it makes the water from 

In [36]:
response = tokenizer.batch_decode(outputs,skip_special_tokens=True)

In [38]:
print(response[0])

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
If its raining why is the sky grey
</problem>
assistant

Hmmm, okay, let me try to understand this. The question is: "If it's raining, why is the sky grey." Hmm, that seems like a pretty straightforward question, but maybe there's more to it. Maybe the sky is grey when it's raining because of how water in the air behaves.

Alright, so first, I remember learning about different types of clouds and how they form. Clouds are made up of water vapor and tiny particles in the air. When the sun heats the Earth, it makes the water from oceans, lakes, and rivers evaporate into the air as water vapor. This process is called evaporation. As the water vapor rises, it cools down, and

In [39]:
# Saving final model weights LORA  and tokenizer
trainer.save_model("outputs/final_model")
tokenizer.save_pretrained("outputs/final_model")

('outputs/final_model/tokenizer_config.json',
 'outputs/final_model/special_tokens_map.json',
 'outputs/final_model/chat_template.jinja',
 'outputs/final_model/tokenizer.json')

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
!cp -r /content/outputs/final_model /content/drive/MyDrive/


In [42]:
!ls /content/drive/MyDrive/final_model

adapter_config.json	   README.md		    tokenizer.json
adapter_model.safetensors  special_tokens_map.json  training_args.bin
chat_template.jinja	   tokenizer_config.json
